In [2]:
from slack_bolt import App
import os
from time import sleep
from hikarie_bot.utils import unix_timestamp_to_jst

ImportError: cannot import name 'init_db' from 'hikarie_bot.database' (/workspaces/hikarie-bot/hikarie_bot/database.py)

In [6]:
app = App(token=os.environ.get("SLACK_BOT_TOKEN"))

In [8]:
# get the channel history of event_notify
channel = "C02CQMLK64C"

response = app.client.conversations_history(channel=channel, limit=200)

In [12]:
data = response.data
messages = []
if data is not None and isinstance(data, dict):
    messages.extend(data["messages"])
    # retrieve all message data
    while data.get("response_metadata") and data["response_metadata"].get(
        "next_cursor"
    ):
        print(
            "pagenating..."
            f'{unix_timestamp_to_jst(data["messages"][0]["ts"]):%Y-%m-%d %H:%M:%S}'
        )
        cursor = data["response_metadata"]["next_cursor"]
        response = app.client.conversations_history(
            channel=channel, cursor=cursor, limit=200
        )
        messages.extend(data["messages"])
        sleep(1)

        next_data = response.data
        if next_data is not None and isinstance(next_data, dict):
            data = next_data

In [18]:
from dataclasses import dataclass
from typing import Any, Self

@dataclass
class SlackMessage:
    user: str
    blocks: list[dict[str, Any]]

    @staticmethod
    def extract_from_response(raw_message:dict) -> Self:
        return SlackMessage(
            user=raw_message["user"],
            blocks=raw_message["blocks"],
        )


In [19]:
messages

[{'user': 'U06A8SJEWK0',
  'type': 'message',
  'ts': '1723634909.949029',
  'bot_id': 'B06AFFYGM2N',
  'app_id': 'A06B4899QSU',
  'text': 'ヒカリエに出社してる？ ',
  'team': 'THDBE7R8R',
  'bot_profile': {'id': 'B06AFFYGM2N',
   'deleted': False,
   'name': 'hikarie-is-god',
   'updated': 1702691709,
   'app_id': 'A06B4899QSU',
   'icons': {'image_36': 'https://a.slack-edge.com/80588/img/plugins/app/bot_36.png',
    'image_48': 'https://a.slack-edge.com/80588/img/plugins/app/bot_48.png',
    'image_72': 'https://a.slack-edge.com/80588/img/plugins/app/service_72.png'},
   'team_id': 'THDBE7R8R'},
  'blocks': [{'type': 'section',
    'block_id': 'BLK_QUESTION',
    'text': {'type': 'mrkdwn', 'text': 'ヒカリエに出社してる？', 'verbatim': False}},
   {'type': 'actions',
    'block_id': 'yhMS+',
    'elements': [{'type': 'button',
      'action_id': 'ACK_ARRIVED_OFFICE',
      'text': {'type': 'plain_text', 'text': '出社した', 'emoji': True}}]}]},
 {'user': 'U06A8SJEWK0',
  'type': 'message',
  'ts': '1723633952.5

In [20]:
SlackMessage.extract_from_response(messages[0])

SlackMessage(user='U06A8SJEWK0', blocks=[{'type': 'section', 'block_id': 'BLK_QUESTION', 'text': {'type': 'mrkdwn', 'text': 'ヒカリエに出社してる？', 'verbatim': False}}, {'type': 'actions', 'block_id': 'yhMS+', 'elements': [{'type': 'button', 'action_id': 'ACK_ARRIVED_OFFICE', 'text': {'type': 'plain_text', 'text': '出社した', 'emoji': True}}]}])